In [1]:
# -- library/semi-analytical.ipynb --
# Author: Jake Cray
# GitHub: crayjake/fgw-python
''' semi-analytical solution '''

' example usage for library '

In [2]:
from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/My Drive/fgw-python
#! git clone 'https://github.com/crayjake/fgw-python.git'
! git pull
#! git checkout .

import sys
sys.path.insert(0,'/content/drive/My Drive/fgw-python/library')

Mounted at /content/drive
/content/drive/My Drive/fgw-python
Already up to date.


In [3]:
import numpy as np
from matplotlib import pyplot as plt


# width      = 1000 * 540 * 1.5 / 2
# spacesteps = 1000

# x = np.linspace(0, width, spacesteps)
# dx = width / spacesteps

In [4]:
from math import floor
from tqdm import tqdm

def trapezium(func, bounds, delta, display=False):
    result = delta * (func(bounds[0]) + func(bounds[1])) / 2

    steps = (bounds[1] - bounds[0]) / delta

    if display:
        for i in tqdm(range(1, floor(steps))):
            result += delta * func(bounds[0] + (delta * i))

    else:
        for i in range(1, floor(steps)):
            result += delta * func(bounds[0] + (delta * i))

    return result
    
def simpson(func, bounds, delta):
    result = delta * (func(bounds[0]) + func(bounds[1])) / 3

    steps = (bounds[1] - bounds[0]) / delta
    for i in range(1, floor(steps)):
        add = 2 * delta * func(bounds[0] + (delta * i))

        if i % 2 != 0:
            add *= 2

        result += add / 3

    return result

In [ ]:
trap = trapezium(lambda x: x ** 2, (0, 10), 0.01)
simp = simpson  (lambda x: x ** 2, (0, 10), 0.01)
print(f'{trap} vs {simp}')

333.33349999999996 vs 333.33333333333337


In [ ]:
from math import sqrt, pi, cos, sin
from cmath import exp

S = 3.6e-5
L = 10000
N = 0.01
rho_s = 1
D = 50000
D_t = 10000
h = 100000

mode = 1

# modal variables
A_j = sqrt(2 / (rho_s * (N ** 2) * D))

# wavespeed (squared)
c2 = ((N * D) ** 2) / (((mode * pi) ** 2) + ((D ** 2)/(4 * (h ** 2)))) # wavespeed

# calculating S_j = S0 * sigma
if ((mode * D_t / D) - 1) == 0:
    S_j = A_j * (rho_s / 2) * D_t
else:
    S_A = np.sin((np.pi) * ((mode * D_t / D) - 1)) / ((mode * D_t / D) - 1)
    S_B = np.sin((np.pi) * ((mode * D_t / D) + 1)) / ((mode * D_t / D) + 1)
    S_j = A_j * (rho_s / 2) * (D_t / np.pi) * (S_A - S_B)

S_j = S_j * S


def F(x):
    return 1 / np.cosh(x / L) ** 2

In [ ]:
steps = 2500

cachedFourier = {}

def FourierTransform(func, k):
    if func in cachedFourier:
        if k in cachedFourier[func]:
            return cachedFourier[func][k]
        
    
    xMax = 50 * L
    dx = xMax / steps
    # e^ix   = cos(x) + i*sin(x)
    # e^-ikx = e^i(-kx) = cos(-kx) + i*sin(-kx) = cos(kx) - i*sin(kx)
    function = lambda x: exp(k * x * -1j) * func(x) 
    value = trapezium(function, (-xMax, xMax), dx)

    if func not in cachedFourier:
        cachedFourier[func] = {}
    cachedFourier[func][k] = value

    return value


def InverseTest(func, x):
    kMax = 30 / L
    dk = kMax / steps

    function = lambda k: exp(k * x * 1j) * func(k)

    return trapezium(function, (-kMax, kMax), dk, True) / (2 * pi) 


In [ ]:
for x in [t * 1000 for t in [1, 2, 5, 25, 50, 150, 300, 405]]:
    true = F(x)

    #fR = lambda k: FourierR(F, k)
    #fI = lambda k: FourierI(F, k)

    #fourier = InverseFourierTransform((fR, fI), x)

    f = lambda k: FourierTransform(F, k)
    fourier = InverseTest(f, x)

    print(f'x={x}: {true} | {fourier} | {str(np.round(100 * (true - fourier) / true, 2))}%')

100%|██████████| 4999/4999 [01:04<00:00, 77.49it/s]


x=1000: 0.9900662908474399 | (0.9900662908474416+5.068812334223141e-17j) | (-0-0j)%


100%|██████████| 4999/4999 [00:00<00:00, 657427.20it/s]


x=2000: 0.9610429829661166 | (0.9610429829661205-1.7480729466439447e-17j) | (-0+0j)%


100%|██████████| 4999/4999 [00:00<00:00, 435467.52it/s]


...

x=150000: 3.743049187535369e-13 | (3.747673921046929e-13+1.326615848725094e-15j) | (-0.12-0.35j)%


100%|██████████| 4999/4999 [00:00<00:00, 276114.75it/s]


x=300000: 3.502604305078608e-26 | (8.130413267013964e-16-9.818364433842948e-16j) | (-2321248008196.36+2803161184838.03j)%


100%|██████████| 4999/4999 [00:00<00:00, 293073.06it/s]

x=405000: 2.6558708798322934e-35 | (-5.588886643863803e-17-1.5458670947024426e-15j) | (2.1043517914608547e+20+5.820565700091781e+21j)%


In [ ]:
a = 5
example      = lambda x: exp(-a * abs(x))
exampleTilde = lambda k: (2 * a) / ((a ** 2) + k ** 2)

testTildeR   = lambda k: FourierR(example, k)
testTildeI   = lambda k: FourierI(example, k)


for k in [0, 1, 5, 10, 50, 100]:
    print(f'Fourier values: (k={k})')
    true    = exampleTilde(k)
    fourier = testTildeR(k) # we know im should be 0

    print(f'{true} | {fourier} | {str(round(100 * (true - fourier) / true, 2))}%')

    print(f'Inverted values: (x={k})')
    true     = example(k)
    inverted = InverseFourierTransform((testTildeR, testTildeI), k)

    print(f'{true} | {inverted} | {str(round(100 * (true - inverted[0]) / true, 2))}%')

    print(f'------------------------------')


Fourier values: (k=0)


NameError: ignored